In [4]:
# !pip install punctuators
# !pip install flair
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [5]:
import torch
from torch import nn
import json
from typing import List
from tqdm import tqdm
import string
import spacy
import re
# from punctuators.models import SBDModelONNX
from augment import AugmentData

# Augment Class

In [6]:

# class AugmentData(nn.Module):
#     def __init__(self, augmentModel="sbd_multi_lang"):
#         super(AugmentData, self).__init__()
#         self.augmentModel = SBDModelONNX.from_pretrained(augmentModel)
#         self.nlp = spacy.load("en_core_web_sm")
#     def build(self):
#         data = self.read_json("config.json")
#     def forward(self, input_dataset_path, output_dataset_path):
#         sentence_pool, span_sentence_pool = self.augment(input_dataset_path)
        
#     def augment(self, input_dataset_path):
#         input_dataset = self.read_json(input_dataset_path)
#         sentence_pool = []
#         span_sentence_pool = []
#         for i in tqdm(range(len(input_dataset))):
#             pool = []
#             span_pool = []
#             for j in range(len(input_dataset[i]['conversation'])):
#                 augment_sentence_pool, augment_sentence_span_pool = self.augment_sentence(input_dataset[i]['conversation'][j]['text'])
#                 pool.append(augment_sentence_pool)
#                 span_pool.append(augment_sentence_span_pool)
            
#             sentence_pool.append(pool)
#             span_sentence_pool.append(span_pool)
#         return sentence_pool, span_sentence_pool
#     def augment_sentence(self, text):
#         sentence_pool = []
#         span_sentence_pool = []

#         unique_list_of_lists, span_list_of_lists = self.split_sentence(text)
#         spacy_pool, spacy_span_pool = self.spacy(text)
#         sentence_pool.extend(unique_list_of_lists)
#         span_sentence_pool.extend(span_list_of_lists)
#         sentence_pool.extend(spacy_pool)
#         span_sentence_pool.extend(spacy_span_pool)
#         sentence_pool, span_sentence_pool = self.remove_duplicate(sentence_pool, span_sentence_pool)
#         return sentence_pool, span_sentence_pool
#     def split_sentence(self, text): 
#         input_texts: List[str] = [text]
#         results: List[List[str]] = self.augmentModel.infer(input_texts)
#         results=results[0]

#         temp = []
#         temp.extend(results)
#         temp.extend(input_texts)
#         unique_list_of_lists = [self.remove_punctuation(x) for x in temp]
#         unique_list_of_lists = [item for item in unique_list_of_lists if len(item)>1]
#         unique_set = set()
#         unique_list_of_lists = [x for x in unique_list_of_lists if tuple(x) not in unique_set and not unique_set.add(tuple(x))]
#         span_list_of_lists = [self.span(input_texts[0], item) for item in unique_list_of_lists]
#         return unique_list_of_lists, span_list_of_lists
#     def split_coordinating_conjunction(self, doc, text):
#         sentence_pool = []
#         span_sentence_pool = []

#         ss = 0
#         words = []
#         text_list = text.split()
#         for i in range(len(doc)):
#             if doc[i].pos_ in ["CCONJ", ]:
#                 se = i+1
#                 sentence_pool.append(" ".join(words))
#                 span_sentence_pool.append((ss, se))
#                 words = []
#                 ss = i+1
#             else:
#                 words.append(text_list[i])
        
#         span = " ".join(words)
#         new_span = self.remove_punctuation(span)
#         ss, se = self.span(text, new_span)
#         sentence_pool.append(new_span)
#         span_sentence_pool.append((ss, se))
#         return sentence_pool, span_sentence_pool
#     def split_comma(self, text):
#         sentence_pool = []
#         span_sentence_pool = []

#         ss = 0
#         words = []
#         text_list = text.split()
#         for i in range(len(text_list)):
#             if text_list[i] == ',':
#                 se = i+1
#                 span = " ".join(words)
#                 new_span = self.remove_punctuation(span)
#                 ss, se = self.span(text, new_span)
#                 sentence_pool.append(new_span)
#                 span_sentence_pool.append((ss, se))
#                 words = []
#                 ss = i+1
#             else:
#                 words.append(text_list[i])
        
#         span = " ".join(words)
#         new_span = self.remove_punctuation(span)
#         ss, se = self.span(text, new_span)
#         sentence_pool.append(new_span)
#         span_sentence_pool.append((ss, se))
#         return sentence_pool, span_sentence_pool
#     def read_json(self, path):
#         with open(path, 'r') as json_file:
#             data = json.load(json_file)
#         return data
#     def write_json(self, path, data):
#         with open('output_json_file.json', 'w') as file:
#             # Write the Python data structure as JSON to the file
#             json.dump(data, file, indent=2)
#     def remove_punctuation(self, text):
#         # Create a string of all punctuation characters
#         punctuation_chars = string.punctuation

#         # Remove punctuation at the beginning and end of the string
#         cleaned_text = text.strip(punctuation_chars)

#         return cleaned_text.strip()
#     def span(self, text, sub_text, s_ind=0):
#         new_text = text.split()
#         new_sub_text = sub_text.split()
#         check = True
#         for i in range(len(new_text)):
#             for j in range(len(new_sub_text)):
#                 if new_sub_text[j] != new_text[i + j]:
#                     check = False
#                     break
#                 else:
#                     temp = i + j
#                     check = True
#             if check == True:
#                 ss = i
#                 se = temp
#                 return s_ind+ss, s_ind+se+1
#     def spacy(self, text):
#         sentence_pool = []
#         span_sentence_pool = []
#         doc = self.nlp(text)

#         comma_pool, comma_span_pool = self.split_comma(text)
#         coordinating_conjunction_pool, coordinating_conjunction_span_pool = self.split_coordinating_conjunction(doc, text)
#         noun_chunks_pool, noun_chunks_span_pool=self.noun_chunks(doc, text)
#         sentence_spacy_pool, sentence_spacy_span_pool = self.sentence_spacy(doc)

#         sentence_pool.extend(comma_pool)
#         span_sentence_pool.extend(comma_span_pool)
#         sentence_pool.extend(coordinating_conjunction_pool)
#         span_sentence_pool.extend(coordinating_conjunction_span_pool)
#         sentence_pool.extend(noun_chunks_pool)
#         span_sentence_pool.extend(noun_chunks_span_pool)
#         sentence_pool.extend(sentence_spacy_pool)
#         span_sentence_pool.extend(sentence_spacy_span_pool)
#         return sentence_pool, span_sentence_pool
#     def noun_chunks(self, doc, text):
#         lll = [doc[i].pos_ for i in range(len(doc))]
#         print(lll)
#         temp = [chunk.text for chunk in doc.noun_chunks]
#         PRON = [token.lemma_ for token in doc if token.pos_ == "PRON"]
#         VERB = [token.lemma_ for token in doc if token.pos_ == "VERB"]
#         values_to_remove = PRON + VERB
#         temp = list(filter(lambda x: x not in values_to_remove, temp))
#         unique_set = set()
#         pool = [self.remove_punctuation(x) for x in temp if tuple(x) not in unique_set and not unique_set.add(tuple(x))]
#         span_pool = [self.span(text, subtext) for subtext in pool]
#         return pool, span_pool
#     def sentence_spacy(self, doc):
#         sentence_pool = []
#         span_sentence_pool = []
#         for sent in doc.sents:
#             sentence_pool.append(str(sent))
#             span_sentence_pool.append((sent.start, sent.end))
#         new_sentence_pool, new_span_sentence_pool = self.concat_sentennce(sentence_pool, span_sentence_pool)
#         return new_sentence_pool, new_span_sentence_pool
#     def concat_sentennce(self, sentence_pool, span_sentence_pool):
#         new_sentence_pool = []
#         new_span_sentence_pool = []
#         for i in range(len(sentence_pool)):
#             for j in range(i, len(sentence_pool)):
#                 ss = span_sentence_pool[i][0]
#                 se = span_sentence_pool[j][1]
#                 span = ' '.join(sentence_pool[i:j+1])
#                 new_span = self.remove_punctuation(span)
#                 ss, se = self.span(span, new_span, ss)
#                 new_sentence_pool.append(new_span)
#                 new_span_sentence_pool.append((ss, se))
#         return new_sentence_pool, new_span_sentence_pool
#     def remove_duplicate(self, sentence_pool, span_sentence_pool):
#         dup_index = [1] * len(sentence_pool)
#         for i in range(len(sentence_pool)-1):
#             for j in range(i+1, len(sentence_pool)):
#                 if sentence_pool[i] == sentence_pool[j]:
#                     dup_index[j] = 0
#         sentence_pool = [sentence_pool[index] for index in range(len(dup_index)) if dup_index[index]]
#         span_sentence_pool = [span_sentence_pool[index] for index in range(len(dup_index)) if dup_index[index]]
#         return sentence_pool, span_sentence_pool

# TEST

In [7]:
ag = AugmentData()
# sentence_pool, span_sentence_pool = ag.augment("/media/s24gb1/90a7e21c-edf4-4782-a0eb-731b73c521c2/DUC/NAACL2024/data/ECF 2.0/test/Subtask_1_test.json")

In [8]:
text = "Not in my head . Anyway , they want to take us out Saturday night ! What do you say ? Umm . Umm . Umm . I do not think so ."
# g.spacy(text)
# l, x = ag.augment_sentence(text)
# for i in range(len(l)):
#     print(l[i])
#     # if len(x[i])>2:
#     print(x[i])
ag.augment_sentence(text, test=True)

[('PART', 'not'), ('ADP', 'in'), ('PRON', 'my'), ('NOUN', 'head'), ('PUNCT', '.'), ('INTJ', 'anyway'), ('PUNCT', ','), ('PRON', 'they'), ('VERB', 'want'), ('PART', 'to'), ('VERB', 'take'), ('PRON', 'we'), ('ADP', 'out'), ('PROPN', 'Saturday'), ('NOUN', 'night'), ('PUNCT', '!'), ('PRON', 'what'), ('AUX', 'do'), ('PRON', 'you'), ('VERB', 'say'), ('PUNCT', '?'), ('INTJ', 'umm'), ('PUNCT', '.'), ('INTJ', 'umm'), ('PUNCT', '.'), ('INTJ', 'umm'), ('PUNCT', '.'), ('PRON', 'I'), ('AUX', 'do'), ('PART', 'not'), ('VERB', 'think'), ('ADV', 'so'), ('PUNCT', '.')]
spacy:['Not in my head . Anyway', 'Not in my head . Anyway , they want to take us out Saturday night ! What do you say ? Umm . Umm . Umm . I do not think so .', 'they want to take us out Saturday night ! What do you say ? Umm . Umm . Umm . I do not think so .', 'Not in my head . Anyway , they want to take us out Saturday night ! What do you say ? Umm . Umm . Umm . I do not think', '', 'my head', 'us', 'What', 'Not in my head .', 'Not in m

(['Not in my head . Anyway , they want to take us out Saturday night ! What do you say ?',
  'Anyway , they want to take us out Saturday night !',
  'Anyway , they want to take us out Saturday night ! What do you say ?',
  'Anyway , they want to take us out Saturday night ! What do you say ? Umm . Umm . Umm . I do not think so .',
  'What do you say ?',
  'Umm . Umm . I do not think so .',
  'Umm . I do not think so .',
  'I do not think so .'],
 [(0, 20), (5, 15), (5, 20), (5, 32), (16, 20), (23, 32), (25, 32), (27, 32)])

In [9]:
text = "Hey ! I did ! But the store would not take her back ! So then I took her to the shelter , and you know what I found out ? If they can not find a home for her , they kill her ! And I am not gonna let that happen to little Yasmine ! Okay , good , good , good , cause , good , cause I was kinda having second thoughts too . Okay . And it is not just chicks you know ? It is all kinds of other animals !"
# g.spacy(text)
# l, x = ag.augment_sentence(text)
# for i in range(len(l)):
#     print(l[i])
#     # if len(x[i])>2:
#     print(x[i])
s, u = ag.augment_sentence(text, test=True)

[('INTJ', 'hey'), ('PUNCT', '!'), ('PRON', 'I'), ('VERB', 'do'), ('PUNCT', '!'), ('CCONJ', 'but'), ('DET', 'the'), ('NOUN', 'store'), ('AUX', 'would'), ('PART', 'not'), ('VERB', 'take'), ('PRON', 'she'), ('ADV', 'back'), ('PUNCT', '!'), ('ADV', 'so'), ('ADV', 'then'), ('PRON', 'I'), ('VERB', 'take'), ('PRON', 'she'), ('ADP', 'to'), ('DET', 'the'), ('NOUN', 'shelter'), ('PUNCT', ','), ('CCONJ', 'and'), ('PRON', 'you'), ('VERB', 'know'), ('PRON', 'what'), ('PRON', 'I'), ('VERB', 'find'), ('ADP', 'out'), ('PUNCT', '?'), ('SCONJ', 'if'), ('PRON', 'they'), ('AUX', 'can'), ('PART', 'not'), ('VERB', 'find'), ('DET', 'a'), ('NOUN', 'home'), ('ADP', 'for'), ('PRON', 'she'), ('PUNCT', ','), ('PRON', 'they'), ('VERB', 'kill'), ('PRON', 'she'), ('PUNCT', '!'), ('CCONJ', 'and'), ('PRON', 'I'), ('AUX', 'be'), ('PART', 'not'), ('VERB', 'go'), ('PART', 'to'), ('VERB', 'let'), ('PRON', 'that'), ('VERB', 'happen'), ('ADP', 'to'), ('ADJ', 'little'), ('PROPN', 'Yasmine'), ('PUNCT', '!'), ('INTJ', 'okay'),

spacy:['Hey ! I did ! But the store would not take her back ! So then I took her to the shelter', 'Hey ! I did ! But the store would not take her back ! So then I took her to the shelter , and you know what I found out ? If they can not find a home for her', 'Hey ! I did ! But the store would not take her back ! So then I took her to the shelter , and you know what I found out ? If they can not find a home for her , they kill her ! And I am not gonna let that happen to little Yasmine ! Okay', 'Hey ! I did ! But the store would not take her back ! So then I took her to the shelter , and you know what I found out ? If they can not find a home for her , they kill her ! And I am not gonna let that happen to little Yasmine ! Okay , good', 'Hey ! I did ! But the store would not take her back ! So then I took her to the shelter , and you know what I found out ? If they can not find a home for her , they kill her ! And I am not gonna let that happen to little Yasmine ! Okay , good , good', 'He

In [10]:
# import spacy

# # Load the English language model
# nlp = spacy.load("en_core_web_sm")

# # Input sentence
# sentence = "You know you probably did not know this , but back in high school , I had a , um , major crush on you ."

# # Process the sentence with spaCy
# doc = nlp(sentence)

# # Split the sentence by pronouns
# split_sentences = []
# current_sentence = []

# for token in doc:
#     # Check if the token is a pronoun
#     if token.pos_ == 'PRON':
#         split_sentences.append(' '.join(current_sentence))
#         current_sentence = [token.text]
#     else:
#         current_sentence.append(token.text)

# # Add the remaining part of the sentence if any
# if current_sentence:
#     split_sentences.append(' '.join(current_sentence))

# # Print the split sentences
# for i, split_sentence in enumerate(split_sentences):
#     print(f"Sentence {i + 1}: {split_sentence}")


In [11]:
!cd GraphEmbedding/
!python setup.py install

/bin/bash: line 1: cd: GraphEmbedding/: No such file or directory


python: can't open file '/media/s24gb1/90a7e21c-edf4-4782-a0eb-731b73c521c2/DUC/NAACL/setup.py': [Errno 2] No such file or directory


In [12]:
# text = "Well , you might try accidentally breaking something valuable of hers , say her"
# ag.spacy(text)

In [13]:
# punctuation_chars = string.punctuation

#         # Remove punctuation at the beginning and end of the string
# cleaned_text = text.strip(punctuation_chars)
# cleaned_text